## 模型性能

In [1]:
import os
import sys
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import *
import numpy as np
sys.path.append('../data/')
sys.path.append("../")
from load_cifar import load_cifar
from load_olivetti import load_olivetti
from resnet import resnet
from preact_resnet import PreActResNet18

In [2]:
os.listdir('ckpts/')

['olivetti_cap_resnet_4_model.ckpt',
 'cifar_cap_res5_0.2_model.ckpt',
 'cifar_cap_res5_0.4_model.ckpt',
 'cifar_no_preact_resnet_model.ckpt',
 'cifar_cap_res5_0.1_model.ckpt',
 'cifar_cap_preact_resnet_0.1_model.ckpt',
 'olivetti_cap_resnet_2_model.ckpt',
 '.ipynb_checkpoints',
 'olivetti_no_res5_model.ckpt',
 'olivetti_cap_resnet_1_model.ckpt',
 'cifar_cap_preact_resnet_0.4_model.ckpt',
 'olivetti_cap_preact_1_model.ckpt',
 'cifar_no_res5_model.ckpt',
 'olivetti_cap_preact_4_model.ckpt',
 'olivetti_cap_preact_2_model.ckpt',
 'olivetti_no_preact_model.ckpt',
 'cifar_cap_preact_resnet_0.2_model.ckpt']

#### CIFAR10/ResNet34

In [3]:
# 加载数据
def reshape_data(X_train, y_train, X_test):
    # reshape train and subtract mean
    pixel_mean = np.mean(X_train, axis=0)
    X_train -= pixel_mean
    X_test -= pixel_mean
    return X_train, y_train, X_test

def load_data(name='cifar'):
    if name == 'cifar':
        X_train, y_train, X_test, y_test = load_cifar(10)
        X_train = np.dstack((X_train[:, :1024], X_train[:, 1024:2048], X_train[:, 2048:]))
        X_train = X_train.reshape((-1, 32, 32, 3)).transpose(0, 3, 1, 2)
        X_test = np.dstack((X_test[:, :1024], X_test[:, 1024:2048], X_test[:, 2048:]))
        X_test = X_test.reshape((-1, 32, 32, 3)).transpose(0, 3, 1, 2)
    else:
        X_train, y_train, X_test, y_test = load_olivetti()
    return X_train, y_train, X_test, y_test

def rbg_to_grayscale(images):
    return np.dot(images[..., :3], [0.299, 0.587, 0.114])

def mal_data_synthesis(train_x, num_targets=10, precision=4):
    num_targets //= 2
    if num_targets == 0:
        num_targets = 1

    targets = train_x[:num_targets]
    input_shape = train_x.shape
    if input_shape[1] == 3:     # rbg to gray scale
        targets = rbg_to_grayscale(targets.transpose(0, 2, 3, 1))

    mal_x = []
    mal_y = []
    for j in range(num_targets):
        target = targets[j].flatten()
        for i, t in enumerate(target):
            t = int(t * 255)
            p = (t - t % (256 // 2 ** precision)) // (2 ** 4)
            p_bits = [p // 2, p - p // 2]
            for k, b in enumerate(p_bits):
                x = np.zeros(input_shape[1:]).reshape(3, -1)
                channel = j % 3
                value = j // 3 + 1.0
                x[channel, i] = value
                if i < len(target) - 1:
                    x[channel, i + 1] = k + 1.0
                else:
                    x[channel, 0] = k + 1.0

                mal_x.append(x)
                mal_y.append(b)

    mal_x = np.asarray(mal_x, dtype=np.float32)
    mal_y = np.asarray(mal_y, dtype=np.int32)
    shape = [-1] + list(input_shape[1:])
    mal_x = mal_x.reshape(shape)
    return mal_x, mal_y, num_targets

In [4]:
def data_to_loader(x, y, size=512):
    x_data = torch.from_numpy(x)
    y_data = torch.from_numpy(y).long()
    loader = DataLoader(dataset=TensorDataset(x_data, y_data), 
                          batch_size=size, 
                          shuffle=False, 
                          num_workers=2)
    return loader

In [5]:
# 加载CIFAR10
X_train, y_train, X_test, y_test = load_data(name='cifar')
mal_x_8, mal_y_8, num_targets = mal_data_synthesis(X_train, num_targets=8)
mal_x_18, mal_y_18, num_targets = mal_data_synthesis(X_train, num_targets=18)
mal_x_38, mal_y_38, num_targets = mal_data_synthesis(X_train, num_targets=38)
X_train, y_train, X_test = reshape_data(X_train, y_train, X_test)
test_loader = data_to_loader(X_test, y_test)
mal_8_loader = data_to_loader(mal_x_8, mal_y_8)
mal_18_loader = data_to_loader(mal_x_18, mal_y_18)
mal_38_loader = data_to_loader(mal_x_38, mal_y_38)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
df_metric = pd.DataFrame(columns=('网络','数据集', '准确率', 'micro-F1', 'macro-F1'))

In [7]:
def metric_model(net, modelpath, data_loader, netname='ResNet34', dataname='CIFAR10'):
    net.load_state_dict(torch.load(modelpath))
    res = []
    y_data = []
    network.eval()
    for i, data in enumerate(data_loader):
        x, y = data
        x_device = x.to(device)
        pred = network(x_device)
        res.append(pred.max(1)[1])
        y_data.append(y.to(device))
    res = torch.cat(res)
    y_data = torch.cat(y_data)
    print(len(res))
    # 准确率
    acc = accuracy_score(res.cpu().numpy(), y_data.cpu().numpy())
    # micro-f1
    micro_f1 = f1_score(res.cpu().numpy(), y_data.cpu().numpy(), average='micro')
    # macro-f1
    macro_f1 = f1_score(res.cpu().numpy(), y_data.cpu().numpy(), average='macro')
    
    global df_metric
    # TODO:修改网络和数据集名字
    df_metric = df_metric.append({
        '网络': netname, 
        '数据集': dataname,
        '准确率': acc,
        'micro-F1': micro_f1,
        'macro-F1': macro_f1
    }, ignore_index=True)
#     df_metric.to_latex(sys.stderr)

In [8]:
# CIFAR10
network = resnet(num_classes=10).to(device)
path = 'ckpts/cifar_cap_res5_0.1_model.ckpt'
metric_model(network, path, test_loader, netname='ResNet34', dataname='CIFAR10')
metric_model(network, path, mal_8_loader, netname='ResNet34', dataname=' synthetic8')
df_metric.to_latex(sys.stderr)

10000
8192


\begin{tabular}{lllrrr}
\toprule
{} &        网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &      CIFAR10 &  0.924400 &  0.924400 &  0.924265 \\
1 &  ResNet34 &   synthetic8 &  0.990601 &  0.990601 &  0.982703 \\
\bottomrule
\end{tabular}


In [9]:
# CIFAR10
network = resnet(num_classes=10).to(device)
path = 'ckpts/cifar_cap_res5_0.2_model.ckpt'
metric_model(network, path, test_loader, netname='ResNet34', dataname='CIFAR10')
metric_model(network, path, mal_18_loader, netname='ResNet34', dataname='synthetic18')
df_metric.to_latex(sys.stderr)

10000
18432


\begin{tabular}{lllrrr}
\toprule
{} &        网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &      CIFAR10 &  0.924400 &  0.924400 &  0.924265 \\
1 &  ResNet34 &   synthetic8 &  0.990601 &  0.990601 &  0.982703 \\
2 &  ResNet34 &      CIFAR10 &  0.919700 &  0.919700 &  0.919553 \\
3 &  ResNet34 &  synthetic18 &  0.998698 &  0.998698 &  0.998184 \\
\bottomrule
\end{tabular}


In [10]:
# CIFAR10
network = resnet(num_classes=10).to(device)
path = 'ckpts/cifar_cap_res5_0.4_model.ckpt'
metric_model(network, path, test_loader, netname='CIFAR10', dataname='CIFAR10')
metric_model(network, path, mal_38_loader, netname='CIFAR10', dataname='synthetic38')
df_metric.to_latex(sys.stderr)

10000
38912


\begin{tabular}{lllrrr}
\toprule
{} &        网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &      CIFAR10 &  0.924400 &  0.924400 &  0.924265 \\
1 &  ResNet34 &   synthetic8 &  0.990601 &  0.990601 &  0.982703 \\
2 &  ResNet34 &      CIFAR10 &  0.919700 &  0.919700 &  0.919553 \\
3 &  ResNet34 &  synthetic18 &  0.998698 &  0.998698 &  0.998184 \\
4 &   CIFAR10 &      CIFAR10 &  0.913000 &  0.913000 &  0.912900 \\
5 &   CIFAR10 &  synthetic38 &  0.999203 &  0.999203 &  0.999237 \\
\bottomrule
\end{tabular}


In [11]:
# CIFAR10
network = PreActResNet18(num_classes=10).to(device)
path = 'ckpts/cifar_cap_preact_resnet_0.1_model.ckpt'
metric_model(network, path, test_loader, netname='PreActResNet18', dataname='CIFAR10')
metric_model(network, path, mal_8_loader, netname='PreActResNet18', dataname='synthetic8')
df_metric.to_latex(sys.stderr)

10000
8192


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &        ResNet34 &      CIFAR10 &  0.924400 &  0.924400 &  0.924265 \\
1 &        ResNet34 &   synthetic8 &  0.990601 &  0.990601 &  0.982703 \\
2 &        ResNet34 &      CIFAR10 &  0.919700 &  0.919700 &  0.919553 \\
3 &        ResNet34 &  synthetic18 &  0.998698 &  0.998698 &  0.998184 \\
4 &         CIFAR10 &      CIFAR10 &  0.913000 &  0.913000 &  0.912900 \\
5 &         CIFAR10 &  synthetic38 &  0.999203 &  0.999203 &  0.999237 \\
6 &  PreActResNet18 &      CIFAR10 &  0.944600 &  0.944600 &  0.944431 \\
7 &  PreActResNet18 &   synthetic8 &  0.999878 &  0.999878 &  0.999423 \\
\bottomrule
\end{tabular}


In [12]:
# CIFAR10
network = PreActResNet18(num_classes=10).to(device)
path = 'ckpts/cifar_cap_preact_resnet_0.2_model.ckpt'
metric_model(network, path, test_loader, netname='PreActResNet18', dataname='CIFAR10')
metric_model(network, path, mal_18_loader, netname='PreActResNet18', dataname='synthetic18')
df_metric.to_latex(sys.stderr)

10000
18432


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &        ResNet34 &      CIFAR10 &  0.924400 &  0.924400 &  0.924265 \\
1 &        ResNet34 &   synthetic8 &  0.990601 &  0.990601 &  0.982703 \\
2 &        ResNet34 &      CIFAR10 &  0.919700 &  0.919700 &  0.919553 \\
3 &        ResNet34 &  synthetic18 &  0.998698 &  0.998698 &  0.998184 \\
4 &         CIFAR10 &      CIFAR10 &  0.913000 &  0.913000 &  0.912900 \\
5 &         CIFAR10 &  synthetic38 &  0.999203 &  0.999203 &  0.999237 \\
6 &  PreActResNet18 &      CIFAR10 &  0.944600 &  0.944600 &  0.944431 \\
7 &  PreActResNet18 &   synthetic8 &  0.999878 &  0.999878 &  0.999423 \\
8 &  PreActResNet18 &      CIFAR10 &  0.944700 &  0.944700 &  0.944635 \\
9 &  PreActResNet18 &  synthetic18 &  1.000000 &  1.000000 &  1.000000 \\
\bottomrule
\end{tabular}


In [13]:
# CIFAR10
network = PreActResNet18(num_classes=10).to(device)
path = 'ckpts/cifar_cap_preact_resnet_0.4_model.ckpt'
metric_model(network, path, test_loader, netname='PreActResNet18', dataname='CIFAR10')
metric_model(network, path, mal_38_loader, netname='PreActResNet18', dataname='synthetic38')
df_metric.to_latex(sys.stderr)

10000
38912


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0  &        ResNet34 &      CIFAR10 &  0.924400 &  0.924400 &  0.924265 \\
1  &        ResNet34 &   synthetic8 &  0.990601 &  0.990601 &  0.982703 \\
2  &        ResNet34 &      CIFAR10 &  0.919700 &  0.919700 &  0.919553 \\
3  &        ResNet34 &  synthetic18 &  0.998698 &  0.998698 &  0.998184 \\
4  &         CIFAR10 &      CIFAR10 &  0.913000 &  0.913000 &  0.912900 \\
5  &         CIFAR10 &  synthetic38 &  0.999203 &  0.999203 &  0.999237 \\
6  &  PreActResNet18 &      CIFAR10 &  0.944600 &  0.944600 &  0.944431 \\
7  &  PreActResNet18 &   synthetic8 &  0.999878 &  0.999878 &  0.999423 \\
8  &  PreActResNet18 &      CIFAR10 &  0.944700 &  0.944700 &  0.944635 \\
9  &  PreActResNet18 &  synthetic18 &  1.000000 &  1.000000 &  1.000000 \\
10 &  PreActResNet18 &      CIFAR10 &  0.937900 &  0.937900 &  0.937839 \\
11 &  PreActResNet18 &  synthetic38 &  1.000000 &  1.00000

## Olivetti

In [14]:
# 加载Olivetti
X_train, y_train, X_test, y_test = load_data(name='olivetti')
mal_x_2, mal_y_2, num_targets = mal_data_synthesis(X_train, num_targets=2)
mal_x_4, mal_y_4, num_targets = mal_data_synthesis(X_train, num_targets=4)
mal_x_8, mal_y_8, num_targets = mal_data_synthesis(X_train, num_targets=8)
X_train, y_train, X_test = reshape_data(X_train, y_train, X_test)
test_loader = data_to_loader(X_test, y_test)
mal_2_loader = data_to_loader(mal_x_2, mal_y_2)
mal_4_loader = data_to_loader(mal_x_4, mal_y_4)
mal_8_loader = data_to_loader(mal_x_8, mal_y_8)

In [15]:
df_metric = pd.DataFrame(columns=('网络','数据集', '准确率', 'micro-F1', 'macro-F1'))

In [16]:
# 2
network = resnet(num_classes=40).to(device)
path = 'ckpts/olivetti_cap_resnet_1_model.ckpt'
metric_model(network, path, test_loader, netname='ResNet34', dataname='Olivetti')
metric_model(network, path, mal_2_loader, netname='ResNet34', dataname=' synthetic2')
df_metric.to_latex(sys.stderr)

120
2048


\begin{tabular}{lllrrr}
\toprule
{} &        网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &     Olivetti &  0.891667 &  0.891667 &  0.882976 \\
1 &  ResNet34 &   synthetic2 &  1.000000 &  1.000000 &  1.000000 \\
\bottomrule
\end{tabular}


In [17]:
# 4
network = resnet(num_classes=40).to(device)
path = 'ckpts/olivetti_cap_resnet_2_model.ckpt'
metric_model(network, path, test_loader, netname='ResNet34', dataname='Olivetti')
metric_model(network, path, mal_4_loader, netname='ResNet34', dataname=' synthetic4')
df_metric.to_latex(sys.stderr)

120
4096


/root/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
\begin{tabular}{lllrrr}
\toprule
{} &        网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &     Olivetti &  0.891667 &  0.891667 &  0.882976 \\
1 &  ResNet34 &   synthetic2 &  1.000000 &  1.000000 &  1.000000 \\
2 &  ResNet34 &     Olivetti &  0.916667 &  0.916667 &  0.913810 \\
3 &  ResNet34 &   synthetic4 &  0.999512 &  0.999512 &  0.874059 \\
\bottomrule
\end{tabular}


In [18]:
# 4
network = resnet(num_classes=40).to(device)
path = 'ckpts/olivetti_cap_resnet_4_model.ckpt'
metric_model(network, path, test_loader, netname='ResNet34', dataname='Olivetti')
metric_model(network, path, mal_8_loader, netname='ResNet34', dataname=' synthetic8')
df_metric.to_latex(sys.stderr)

120
8192


\begin{tabular}{lllrrr}
\toprule
{} &        网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &     Olivetti &  0.891667 &  0.891667 &  0.882976 \\
1 &  ResNet34 &   synthetic2 &  1.000000 &  1.000000 &  1.000000 \\
2 &  ResNet34 &     Olivetti &  0.916667 &  0.916667 &  0.913810 \\
3 &  ResNet34 &   synthetic4 &  0.999512 &  0.999512 &  0.874059 \\
4 &  ResNet34 &     Olivetti &  0.933333 &  0.933333 &  0.930595 \\
5 &  ResNet34 &   synthetic8 &  0.999512 &  0.999512 &  0.997100 \\
\bottomrule
\end{tabular}


In [19]:
# 2/pre
network = PreActResNet18(num_classes=40).to(device)
path = 'ckpts/olivetti_cap_preact_1_model.ckpt'
metric_model(network, path, test_loader, netname='PreActResNet18', dataname='Olivetti')
metric_model(network, path, mal_2_loader, netname='PreActResNet18', dataname=' synthetic2')
df_metric.to_latex(sys.stderr)

120
2048


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &        ResNet34 &     Olivetti &  0.891667 &  0.891667 &  0.882976 \\
1 &        ResNet34 &   synthetic2 &  1.000000 &  1.000000 &  1.000000 \\
2 &        ResNet34 &     Olivetti &  0.916667 &  0.916667 &  0.913810 \\
3 &        ResNet34 &   synthetic4 &  0.999512 &  0.999512 &  0.874059 \\
4 &        ResNet34 &     Olivetti &  0.933333 &  0.933333 &  0.930595 \\
5 &        ResNet34 &   synthetic8 &  0.999512 &  0.999512 &  0.997100 \\
6 &  PreActResNet18 &     Olivetti &  0.966667 &  0.966667 &  0.965952 \\
7 &  PreActResNet18 &   synthetic2 &  0.989746 &  0.989746 &  0.980542 \\
\bottomrule
\end{tabular}


In [20]:
# 2/pre
network = PreActResNet18(num_classes=40).to(device)
path = 'ckpts/olivetti_cap_preact_2_model.ckpt'
metric_model(network, path, test_loader, netname='PreActResNet18', dataname='Olivetti')
metric_model(network, path, mal_4_loader, netname='PreActResNet18', dataname=' synthetic4')
df_metric.to_latex(sys.stderr)

120
4096


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &        ResNet34 &     Olivetti &  0.891667 &  0.891667 &  0.882976 \\
1 &        ResNet34 &   synthetic2 &  1.000000 &  1.000000 &  1.000000 \\
2 &        ResNet34 &     Olivetti &  0.916667 &  0.916667 &  0.913810 \\
3 &        ResNet34 &   synthetic4 &  0.999512 &  0.999512 &  0.874059 \\
4 &        ResNet34 &     Olivetti &  0.933333 &  0.933333 &  0.930595 \\
5 &        ResNet34 &   synthetic8 &  0.999512 &  0.999512 &  0.997100 \\
6 &  PreActResNet18 &     Olivetti &  0.966667 &  0.966667 &  0.965952 \\
7 &  PreActResNet18 &   synthetic2 &  0.989746 &  0.989746 &  0.980542 \\
8 &  PreActResNet18 &     Olivetti &  0.900000 &  0.900000 &  0.896905 \\
9 &  PreActResNet18 &   synthetic4 &  0.998047 &  0.998047 &  0.995443 \\
\bottomrule
\end{tabular}


In [21]:
# 2/pre
network = PreActResNet18(num_classes=40).to(device)
path = 'ckpts/olivetti_cap_preact_4_model.ckpt'
metric_model(network, path, test_loader, netname='PreActResNet18', dataname='Olivetti')
metric_model(network, path, mal_8_loader, netname='PreActResNet18', dataname=' synthetic8')
df_metric.to_latex(sys.stderr)

120
8192


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &          数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0  &        ResNet34 &     Olivetti &  0.891667 &  0.891667 &  0.882976 \\
1  &        ResNet34 &   synthetic2 &  1.000000 &  1.000000 &  1.000000 \\
2  &        ResNet34 &     Olivetti &  0.916667 &  0.916667 &  0.913810 \\
3  &        ResNet34 &   synthetic4 &  0.999512 &  0.999512 &  0.874059 \\
4  &        ResNet34 &     Olivetti &  0.933333 &  0.933333 &  0.930595 \\
5  &        ResNet34 &   synthetic8 &  0.999512 &  0.999512 &  0.997100 \\
6  &  PreActResNet18 &     Olivetti &  0.966667 &  0.966667 &  0.965952 \\
7  &  PreActResNet18 &   synthetic2 &  0.989746 &  0.989746 &  0.980542 \\
8  &  PreActResNet18 &     Olivetti &  0.900000 &  0.900000 &  0.896905 \\
9  &  PreActResNet18 &   synthetic4 &  0.998047 &  0.998047 &  0.995443 \\
10 &  PreActResNet18 &     Olivetti &  0.891667 &  0.891667 &  0.888214 \\
11 &  PreActResNet18 &   synthetic8 &  0.999634 &  0.99963